In [36]:
import requests
from bs4 import BeautifulSoup
import json
import numpy as np

In [2]:
def gen_data(quote):
    quoteText = quote.select_one("div.quoteText")
    text = quoteText.text.split("\n")
    author = text[-3].strip()
    quote_text = text[1].strip()
    
    footer = quote.select_one("div.quoteFooter")
    tags_list = footer.select_one("div.greyText")
    tags = []
    if tags_list:
        for tag in tags_list.find_all("a"):
            tags.append(tag.text)
    
    deatails = {}
    deatails["author"] = author
    deatails["quote"] = quote_text
    deatails["tags"] = tags
    
    return deatails

In [3]:
def grab(url):
    data = requests.get(url).text
    soup = BeautifulSoup(data)
    quotes_data = soup.find_all("div", attrs={"class" : "quote"})
    quotes = []
    
    for q in quotes_data:
        quotes.append(gen_data(q))
    
    return quotes
# gen_data(quote)

In [80]:
emotions = ["love", "humor", "education"]

In [5]:
base = "https://www.goodreads.com/quotes/tag/{0}?page={1}"

for e in emotions:
    all_quotes = []
    for i in range(1, 4):
        print("scrapping page " + str(i))
        all_quotes += grab(base.format(e, str(i)))
    json.dump(all_quotes, open(e+".json", "w", encoding="utf16"))  
        

scrapping page 1
scrapping page 2
scrapping page 3
scrapping page 1
scrapping page 2
scrapping page 3
scrapping page 1
scrapping page 2
scrapping page 3


In [81]:
X = []
y = []

for index, e in enumerate(emotions):
    quotes = json.load(open(e + ".json", "r", encoding="utf16"))
    lines = [q["quote"][1:-1] for q in quotes]
    X += lines
    y += [index]*len(lines)

In [120]:
from nltk import word_tokenize
from nltk import sent_tokenize
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer

In [83]:
sw = set(stopwords.words("english"))
punc = set(string.punctuation)
useless = sw.union(punc)

pts = PorterStemmer()

In [84]:
def mytokenizer(line):
    words = word_tokenize(line.lower())
    usefull = [w for w in words if w not in useless]
    res = [pts.stem(w) for w in usefull]
    return res

In [122]:
cv = CountVectorizer(tokenizer=mytokenizer, ngram_range=(1, 2), min_df=.007, max_df=.09)

In [123]:
vec = cv.fit_transform(X)

vals = vec.todense()

In [124]:
len(cv.vocabulary_)

676

In [125]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB

In [126]:
X_train, X_test, y_train, y_test = train_test_split(vals, y, test_size=0.33)

In [127]:
nb = MultinomialNB()

In [128]:
nb.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [129]:
nb.score(X_test, y_test)

0.5222222222222223